In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import flet as ft
from flet import (
    Container,
    Icon,
    Page,
    Text,
    AppBar,
    PopupMenuButton,
    PopupMenuItem,
    colors,
    icons,
    margin
)

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id='', client_secret='')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
#Finding the ID of the song from song name and artist name
def get_song_id(song_name, artist_name):
    results = sp.search(q=f'track:{song_name} artist:{artist_name}', limit=1, type='track')
    if results['tracks']['items'] == []:
        return None
    return results['tracks']['items'][0]['id']

In [5]:
# Getting the features from spotify
def get_song_features(song_id, num_songs=100):
    #Getting the features of the target song
    target_features = sp.audio_features(song_id)
    if target_features[0] is None:
        return None, None
    target_features_df = pd.DataFrame(target_features)

    #Getting 100 songs from spotify
    recommended_tracks = sp.recommendations(seed_tracks=[song_id], limit=num_songs)['tracks']

    #Getting features for each
    recommended_features = []
    for track in recommended_tracks:
        features = sp.audio_features(track['id'])
        if features[0] is not None:
            recommended_features.append(features[0])
    recommended_features_df = pd.DataFrame(recommended_features)

    #Returning
    return target_features_df, recommended_features_df

In [6]:
#Recommendation Method
def recommend_songs(song_name, artist_name, num_recommendations=5):
    #Get ID
    song_id = get_song_id(song_name, artist_name)
    if song_id is None:
        print(f"No track found for {song_name} by {artist_name}")
        return

    #Get features method
    target_features_df, recommended_features_df = get_song_features(song_id)
    if target_features_df is None or recommended_features_df is None:
        print(f"No audio features found for {song_name} by {artist_name}")
        return

    #Cosine Similarity calculation
    cosine_similarities = cosine_similarity(target_features_df.iloc[:, :11], recommended_features_df.iloc[:, :11])

    #Getting the indices of the best ones
    top_indices = np.argsort(cosine_similarities[0])[:-num_recommendations-1:-1]

    top_tracks = []

    #Screen print for correction check
    for i, index in enumerate(top_indices):
        track = sp.track(recommended_features_df.iloc[index]['id'])
        print(f"Recommendation #{i + 1}: {track['name']} by {track['artists'][0]['name']}")
        top_tracks.append(f"{track['name']} by {track['artists'][0]['name']}")


    print(top_tracks)
    return top_tracks


recommend_songs('Shiver', 'Coldplay')

Recommendation #1: Wonderful by Stone Temple Pilots
Recommendation #2: Island In The Sun by Weezer
Recommendation #3: Regret - 2015 Remaster by New Order
Recommendation #4: Golden Cage by The Whitest Boy Alive
Recommendation #5: Blind by Lifehouse
['Wonderful by Stone Temple Pilots', 'Island In The Sun by Weezer', 'Regret - 2015 Remaster by New Order', 'Golden Cage by The Whitest Boy Alive', 'Blind by Lifehouse']


['Wonderful by Stone Temple Pilots',
 'Island In The Sun by Weezer',
 'Regret - 2015 Remaster by New Order',
 'Golden Cage by The Whitest Boy Alive',
 'Blind by Lifehouse']

In [1]:
#Basic UI
from flet import *
import flet



def manage(page:flet.Page):

    page.window_resizable = False
    page.bgcolor = 'Black'
    page.padding = 0
    

    
    intro = Container(
        content=Column(
            alignment=flet.alignment.center,
            controls = [
                Row(
                    controls=[
                        Container(
                            content=Icon(
                                icons.MENU
                            )
                        )
                    ]
                ),
            Container(height=20),
            Text(
                value='Rosrid Music Recommender, where dreams come true!',
                style= flet.TextThemeStyle.DISPLAY_MEDIUM
            ),
            Text(
                value='To use the program, you are required to give a song name and an artist name, that is all, simple right?',
            ),
        ]
    )
)

    def get_song(p):
        song_name.value = song.value
        artist_name.value = artist.value
        result.value = recommend_songs(song.value, artist.value)
        page.update()

        
    
    

    song_name = flet.Text("")
    artist_name = flet.Text("")
    result = flet.Text("")
    songRow = flet.Row(
        controls=[
            song_name,
            artist_name,
        ],
        alignment=flet.MainAxisAlignment.START
    )

        
    song = flet.TextField(label="song")
    artist = flet.TextField(label="artist")
    button = flet.ElevatedButton("Click me for new songs!", width=300, on_click=get_song, icon="QUEUE_MUSIC", icon_color="BLACK")

    boxes = Container(
        content=Column(
            controls=[
                song,
                artist,
                button,
                songRow,
                result
            ]
        ),
    )



    texts = Row(
        alignment=flet.alignment.center,
        controls=[
            Container(
                alignment=alignment.center,
                width=1290,
                height=720,
                bgcolor=flet.colors.BLACK12,
                border_radius=35,
                padding=padding.only(
                    top=100,left=80,
                    right=20, bottom=5
                ),
                content=Column(
                    alignment=flet.alignment.center,
                    controls=[
                        intro,
                        boxes,

                    ]
                )
            )
        ]
    )

    colorbg = Container(

        width = 1280,
        height = 720,
        gradient=flet.RadialGradient(
        colors=[flet.colors.BLUE_GREY_900, flet.colors.INDIGO_100],
        ),
        border_radius=200,
        content=Stack(
            controls = [
                    texts
                ]
            )
    ) 
    
    
    
    page.add(colorbg)

flet.app(target=manage)